# Challenge 1

Oppgaven er å lage et program (liste av (qssL/Rq)-instruksjoner for en variant av COPY-funksjonen:

Input er en endelig streng  av1-ere og 0-ere. 
Strengen er vilkårlig lang. I figuren vises kun et eksempel på en 7-bits streng.
Algoritmen skal fungere uavhengig av hvor lang inputstrengen er
Oppgaven er å kopiere denne input strengen rett til venstre for strengen, som vist i figuren. Det er ikke viktig hvilken celle algoritmen slutter i.

![alt text](Problem_image.PNG)


Tips:

Anta algoritmen starter helt til venstre av inputstrengen. Denne tilstanden kaller vi q0, dvs. starttilstanden. 
Symbolene på tapen er 0,1,b, hvor b står for blank og skrives ikke på figuren av syns skyld.
Du skal definere ett sett med tilstander q1,q2,q3,… Hvor mange tilstander du bruker er helt opp til deg. Jo flere tilstander du bruker, desto «vanskeligere» blir koden å forstå. 

For hver tilstand du definerer må du avgjøre hva du skal gjøre hvis det avleste symbolet er 0,1 eller b.
Sluttilstanden skrives bare som e
Det hjelper å tenke på en tilstand som en «enkel strategi» som du trenger for ¨løse problemet, f.eks. «gå helt til høyre for strengen», «kopier en 1 helt til venstre for strengen» etc. men nå har jeg sagt altfor mye allerede hehe 

# Opplastede løsninger fra studenter

Vi har fått inn løsning fra to studenter:

## Andreas Rovik

In [ ]:
#!C:\Python310\python.exe

from time import sleep

class bcolors:
    WHITE = '\033[0m'
    RED = '\033[31m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class TuringMachine:

    # List simulating the Turing machine tape, where each index represents 
    # a cell on the tape.
    TAPE = None

    # The cell on the tape which the machine head points to when the
    # program is executing instructions.
    POINTER = None

    # States modifying the instructions to be executed as the head pointer
    # moves into and reads a cell.
    STATES = None

    # Which instructions the Turing machine will execute when called.
    CURR_STATE = None

    # Did the program finish executing all instructions until completion?
    # Progam halts when instruction executes state shift to "E" or "reject".
    HALT = False

    # Number of steps required to complete algorithm.
    STEPS = 0

    # Number of empty cells to render on each side of the tape.
    # By default tape is set to '2 x input string' and will render additonal
    # cells as necessary.
    PADDING = 2

    def __init__(
        self,
        start_cell: int,
        start_string: str,
        states: dict,
        initial_state: str,
    ) -> None:
        self.POINTER = start_cell + len(start_string) + self.PADDING
        self.TAPE = [" "] * (len(start_string) + self.PADDING)
        self.TAPE.extend([cell for cell in start_string])
        self.TAPE.extend([" "] * self.PADDING)
        self.STATES = states
        self.CURR_STATE = initial_state
        self.render()

    def execute(self) -> None:
        self.STEPS += 1

        symbol = self.TAPE[self.POINTER]
        instructions = self.STATES[self.CURR_STATE][symbol]

        to_write = instructions[0]
        to_move = instructions[1]
        next_state = instructions[2]

        if to_write:
            self.TAPE[self.POINTER] = to_write

        if to_move:
            self.POINTER = self.POINTER + (1 if to_move == "R" else -1)

            if self.POINTER == -1:
                self.TAPE.insert(0, " ")
                self.POINTER = 0

            elif self.POINTER > len(self.TAPE) - 1:
                self.TAPE.append(" ")

        self.CURR_STATE = next_state
        if next_state == "reject": # error
            self.HALT = True
        if next_state == "E": # Halt
            self.HALT = True
        return

    # Show the user some graphic to visualize the Turing machine program.
    def render(self) -> None:
        sleep(0.075)
        colored_tape = self.TAPE.copy()
        colored_value = colored_tape[self.POINTER]

        if colored_value == " ":
            colored_value = "B"

        colored_tape[self.POINTER] = bcolors.RED + colored_value + bcolors.WHITE + bcolors.BOLD
        print(bcolors.BOLD + f"| {' | '.join(colored_tape)} |\t[{self.CURR_STATE}]" + bcolors.ENDC)

    def print_stats(self) -> None:
        print(bcolors.RED + f"\n\nTotal of {self.STEPS} steps, at {len(self.STATES)} states." + bcolors.ENDC)

    # Executes one instruction from the tape and renders the result.
    def step(self):
        if not self.HALT:
            self.execute()
            self.render()

    # Executes instructions until halt (or error).
    def start(self):
        while not self.HALT:
            self.execute()
            self.render()
        else:
            self.print_stats()


states = {    
    "qEntry": {
        "0": [None, "R", "qEntry"],
        "1": [None, "R", "qEntry"],
        " ": [None, "L", "qCopyLeft"],
    },

    "qCopyLeft": {
        "0": [" ", "L", "qSendLeft0"],
        "1": [" ", "L", "qSendLeft1"],
        " ": [None, None, "qShiftRight"],
    },

    "qSendLeft0": {
        "0": [None, "L", "qSendLeft0"],
        "1": [None, "L", "qSendLeft0"],
        " ": [None, "L", "qInsert0"],
    },
    "qSendLeft1": {
        "0": [None, "L", "qSendLeft1"],
        "1": [None, "L", "qSendLeft1"],
        " ": [None, "L", "qInsert1"],
    },

    "qInsert0": {
        "0": [None, "L", "qInsert0"],
        "1": [None, "L", "qInsert0"],
        " ": ["0", "R", "qSendRight0"],
    },
    "qInsert1": {
        "0": [None, "L", "qInsert1"],
        "1": [None, "L", "qInsert1"],
        " ": ["1", "R", "qSendRight1"],
    },

    "qSendRight0": {
        "0": [None, "R", "qSendRight0"],
        "1": [None, "R", "qSendRight0"],
        " ": [None, "R", "qReturn0"],
    },
    "qSendRight1": {
        "0": [None, "R", "qSendRight1"],
        "1": [None, "R", "qSendRight1"],
        " ": [None, "R", "qReturn1"],
    },

    "qReturn0": {
        "0": [None, "R", "qReturn0"],
        "1": [None, "R", "qReturn0"],
        " ": ["0", "L", "qCopyLeft"],
    },
    "qReturn1": {
        "0": [None, "R", "qReturn1"],
        "1": [None, "R", "qReturn1"],
        " ": ["1", "L", "qCopyLeft"],
    },

    "qShiftRight": {
        "0": [None, None, "reject"],
        "1": [None, None, "reject"],
        " ": [None, "L", "qShift"],
    },
    "qShift": {
        "0": [" ", "R", "qShift0"],
        "1": [" ", "R", "qShift1"],
        " ": [None, None, "E"],
    },
    "qShift0": {
        "0": [None, None, "reject"],
        "1": [None, None, "reject"],
        " ": ["0", "L", "qShiftRight"],
    },
    "qShift1": {
        "0": [None, None, "reject"],
        "1": [None, None, "reject"],
        " ": ["1", "L", "qShiftRight"],
    },
}

x = TuringMachine(0, "1010011", states, "qEntry")
x.start()



## HaukeSL (Dircord)

In [ ]:
"""
Oppgave: lage et program som kan kjøre algoritmer til Turing Machine 
1. Definere et sett med tilstander til en algoritme (Oppgave 30.8.2022: copy algoritme)
2. Definere instruksjoner av hver tilstand.
3. Kode imitation av TM steg for steg. Må fungere med arbitrary queries og input.
4. Kjøre algoritmen i TM programmet.

OBS! Jeg har ikke noe copy algoritme enda. Men programmet fungerer med double ones. 
"""

print("\033c")  # clears everything on console output

##### states first #####
# L/R variables as integers to be applied on tape index
L = - 1
R = + 1

# states for a copy function, as dictionaries:
# q-example = {"s-read":[s-print,L/R,q-next]}
# " " = blank 
# if next q is = -1, it means exit state. At the exit, L/R is practically not needed for a TM. Keeping the L/R just helps with the structure and tidiness of the dictionary.

# q0 = starting state & finds last, most-right number
q0 = {
    "1": ["1", R, 0],
    "0": ["0", R, 0],
    " ": [" ", L, 1]
}

# q1 =  takes value 1 or 0, blanks, moves right
q1 = {
    "1": [" ", R, 2],
    "0": [" ", R, 3],
    " ": [" ", L, -1]
}

# q2 Prints 1, prepares for final loop with 1: | | | |0|1|1| |1| | | |
q2 = {
    "1": [" ", L, -1],
    "0": [" ", L, -1],
    " ": ["1", L, 4]
}

# q3 Prints 0, prepares for final loop with 0: | | | |0|1|1| |0| | | |
q3 = {
    "1": [" ", L, -1],
    "0": [" ", L, -1],
    " ": ["0", L, 4]
}

# q4 Jumps back to second last value on tape, we are ready to copy now
q4 = {
    "1": [" ", R, -1],
    "0": [" ", R, -1],
    " ": [" ", L, 5]
}

# q5 starting copy process of either 1 or 0
q5 = {
    "1": [" ", L, 6],
    "0": [" ", L, 7],
    " ": [" ", R, 16]
}

# a seperation blank will serve as a placeholder between copy and original. It is the  field for the last number to the right
# q6= moves left to first blank, ("keeps" 1) -> creates a blank between org and copy string now |0|0| |1|1|
q6 = {
    "1": ["1", L, 6],
    "0": ["0", L, 6],
    " ": [" ", L, 8]
}

# q7 = moves left to first blank, ("keeps" 0) -> creates a blank between org and copy string now | |1| |1|1|

q7 = {
    "1": ["1", L, 7],
    "0": ["0", L, 7],
    " ": [" ", L, 9]
}

# q8 = from blank, moves left, prints 1
q8 = {
    "1": ["1", L, 8],
    "0": ["0", L, 8],
    " ": ["1", R, 10]
}

# q9 = from blank, moves left, prints 0
q9 = {
    "1": ["1", L, 9],
    "0": ["0", L, 9],
    " ": ["0", R, 11]
}

# q10 = moves right, meets seperation blank ("keeps" 1)
q10 = {
    "1": ["1", R, 10],
    "0": ["0", R, 10],
    " ": [" ", R, 14]
}

# q11 = moves right, meets seperation blank ("keeps" 0)
q11 = {
    "1": ["1", R, 11],
    "0": ["0", R, 11],
    " ": [" ", R, 15]
}
# if the seperation blank is followed up by a blank, the exit sequence initiates (q16-q23)
# q12 = continues from sep blank right ("keeps" 1), checks for possible end of input: | |1|1| | |0|0|1|1| |1
q12 = {
    "1": ["1", R, 14],
    "0": ["0", R, 14],
    " ": [" ", R, 16]
}

# q13 = continues from sep blank right ("keeps" 0), checks for possible end of input: | |1|1| | |0|0|1|1| |0
q13 = {
    "1": ["1", R, 15],
    "0": ["0", R, 15],
    " ": [" ", R, 17]
}

# q14 = from sep blank, moves right, prints 1
q14 = {
    "1": ["1", R, 14],
    "0": ["0", R, 14],
    " ": ["1", L, 5]
}

# q15 = from sep blank, moves right, prints 0
q15 = {
    "1": ["1", R, 15],
    "0": ["0", R, 15],
    " ": ["0", L, 5]
}

# q16-23 are only for the last copy and to exit the loop
# q16 = moves right, "keep" 1, initiates final q's (if last value is 1)
q16 = {
    "1": ["1", R, 16],
    "0": ["0", R, 16],
    " ": [" ", R, 18]
}

# q17 = moves right, keeps "0", initiates final q's (if last value is 0)
q17 = {
    "1": ["1", R, 17],
    "0": ["0", R, 17],
    " ": [" ", R, 18]
}

# q18 = blanks out the right most field behind the placeholder blank
q18 = {
    "1": [" ", L, 19],
    "0": [" ", L, 20],
    " ": [" ", L, -1]
}
# q19 prints 1 on placeholder blank, the last right number is placed correctly now
q19 = {
    "1": ["1", L, 19],
    "0": ["0", L, 19],
    " ": ["1", L, 21]
}

# q20 prints 0 on placeholder blank, the last right number is placed correctly now
q20 = {
    "1": ["1", L, 20],
    "0": ["0", L, 20],
    " ": ["0", L, 22]
}

# q21 moves left to the seperation blank, prints 1
q21 = {
    "1": ["1", L, 21],
    "0": ["0", L, 21],
    " ": ["1", L, 23]
}

# q22 moves left to the seperation blank, prints 0
q22 = {
    "1": ["1", L, 22],
    "0": ["0", L, 22],
    " ": ["0", L, 23]
}

# q23 the while loop needs an extra query, otherwise it ends before printing the last char.
q23 = {
    "1": ["1", R, -1],
    "0": ["0", R, -1],
    " ": [" ", R, -1]
}

# list of all state dictionaries, which will help us to find the correct next state
states = [q0, q1, q2, q3, q4, q5, q6, q7, q8, q9, q10, q11,
          q12, q13, q14, q15, q16, q17, q18, q19, q20, q21, q22, q23]

# list of states as strings for possible printing purposes
str_states = ["q0", "q1", "q2", "q3", "q4", "q5",
              "q6", "q7", "q8", "q9", "q10", "q11", "q12", "q13",
              "q14", "q15", "q16", "q17", "q18", "q19", "q20", "q21", "q22", "e"]

#### states end ####

#input function
def tape_create():
    user_input = input("Enter input: ")
    tape_list = [" ", " ", " ", " "]
    i = 0
    while i < len(user_input):
        if user_input[i] != "0" and user_input[i] != "1":
            user_input = input("Wrong value detected. Enter only 0s and/or 1s. Try again: ")
            i = 0
        else:
            tape_list.insert(-3, user_input[i])
            tape_list.insert(0, " ")

        i += 1

    return tape_list


# startcell function. if statecell = first non-blank value.
def find_start(tape: list):
    """
    Case: start cell at first non-blank value
    input: tape
    output: returns index of first non-blank value on the tape array
    """
    if len(tape) > 0:
        for i in range(len(tape)):
            if tape[i] != " ":
                return i
    return 0

#function to print pretty console output
class bcolors:
    OKBLUE = '\033[94m'  # blue
    OKGREEN = '\033[92m'  # green
    RED = '\033[91m'  # red
    ENDC = '\033[0m'  # white

def visual_head_moves(tape: list, curr_pos: int, curr_val: int, instruc_val: int) -> None: 
    string = ""
    colortape = list(tape)

    for j in range(0, len(colortape)):
        if j == curr_pos:
            if curr_val != instruc_val:
                    if curr_val == " ":
                        string = string+"|"+ bcolors.RED + "B"+ bcolors.ENDC
                    else:
                        string = string+"|"+ bcolors.RED + colortape[j]+ bcolors.ENDC
            else:
            
                if curr_val == " ":
                    string = string+"|"+ bcolors.OKBLUE+ "B"+ bcolors.ENDC 
                else:
                    string = string+"|"+ bcolors.OKBLUE + colortape[j]+ bcolors.ENDC
        else:
            string = string +"|"+ colortape[j]

    print(string)


# Turing machine programm

def turingmachine(instructions: list, tape: list, ) -> list:
    """
    Input: list of states, tape, start position.
    Output: altered tape
    Method: while loop, making each step equivalent to a Turing Machine, the sequence is altered to fit the input and the while loop
    """
    step_counter = 0
    curr_state = instructions[0]  # start state
    curr_pos = find_start(tape)  # start position
    curr_val = tape[curr_pos]  # value at start position
    instruc_val = curr_state[curr_val][0]


    # while loop stops if next state is -1
    while curr_state[curr_val][2] != -1:

        visual_head_moves(tape,curr_pos,curr_val,instruc_val)
        # writes instructed value to tape
        tape[curr_pos] = instruc_val

        # continues to next position as instructed(L/R)
        curr_pos += curr_state[curr_val][1]

        # continues with instructed next state
        curr_state = instructions[curr_state[curr_val][2]]

        # reads value of the current position (1,0,b)
        curr_val = tape[curr_pos]

        # updating the instruction value from current state
        instruc_val = curr_state[curr_val][0]

        # step counter increment
        step_counter += 1

    visual_head_moves(tape,curr_pos,curr_val,instruc_val)
    print(bcolors.OKBLUE,"\nTotal of", step_counter,"steps.",bcolors.ENDC)
    
    return tape





#input tape as list of strings, string starts here at pos 10
input_copy_tape = [ " ", " ", " ", " ", " ", " ", " ", " ",
              "1", "0", "1", "0", "0", "1", "1",
              " ", " ", " "]

turingmachine(states,tape_create())